# **About**

This is a simple demonstration of implementing Neural Network using PyTorch.

Basically this network echoes the input text that is given.

Goal : =>  `'this is a test' => [Neural Network] => 'this is a test' `

In [0]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# **Data**

In [0]:
trainData = '''Lorem Ipsum is simply dummy text of the printing and typesetting industry Lorem Ipsum has been the industrys
  standard dummy text ever since the when an unknown printer took a galley of type and scrambled it to make
  a type specimen book It has survived not only five centuries but also the leap into electronic typesetting remaining
  essentially unchanged It was popularised in the with the release of Letraset sheets containing Lorem Ipsum passages
  and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum Why do we use
  it It is a long established fact that a reader will be distracted by the readable content of a page when looking at its
  layout The point of using Lorem Ipsum is that it has a moreorless normal distribution of letters as opposed to using
  Content here content here making it look like readable English Many desktop publishing packages and web page
  editors now use Lorem Ipsum as their default model text and a search for lorem ipsum will uncover many web
  sites still in their infancy Various versions have evolved over the years sometimes by accident sometimes
  on purpose injected humour and the like Where does it come from Contrary to popular belief Lorem Ipsum is not
  simply random text It has roots in a piece of classical Latin literature from BC making it over years old
  Richard McClintock a Latin professor at Hampden Sydney College in Virginia looked up one of the more obscure
  Latin words consectetur from a Lorem Ipsum passage and going through the cites of the word in classical
  literature discovered the undoubtable source Lorem Ipsum comes from sections  and  of de
  Finibus Bonorum et Malorum The Extremes of Good and Evil by Cicero written in BC This book is a
  treatise on the theory of ethics very popular during the Renaissance The first line of Lorem Ipsum Lorem
  ipsum dolor sit amet comes from a line in section The standard chunk of Lorem Ipsum used
  since the is reproduced below for those interested Sections and from de Finibus
  Bonorum et Malorum by Cicero are also reproduced in their exact original form accompanied by English
  versions from the translation by H Rackham Our loss function is what calculates how far off our classifications are from reality
  As humans we tend to think of things in terms of either right or wrong With a neural network and arguably humans too
  our accuracy is actually some sort of scaling score For example you might be highly confident that something is the case
  but you are wrong Compare this to a time when you really arent certain either way but maybe think something but are wrong 
  In these cases the degree to which youre wrong doesnt matter in terms of the choice necessarily but in terms of you learning it does
  In terms of a machine learning by tweaking lots of little parameters to slowly get closer and closer to fitting it 
  definitely matters how wrong things are For this we use loss which is a measurement of how far off the neural network 
  is from the targeted output There are a few types of loss calculations A popular one is mean squared error but were
  trying to use these scalar valued classes In general youre going to have two types of classes One will
  just be a scalar value the other is whats called a onehot array vector'''

This paragraph is split into words and saved in `trainset`

In [0]:
trainset = trainData.split()
testset = ['hello','bye','see you','later']

# **Data PreProcessing functions**

In [0]:
charTobin = {
            'a': [0,0,0,0,0],'b': [0,0,0,0,1],'c': [0,0,0,1,0],'d': [0,0,0,1,1],
            'e': [0,0,1,0,0],'f': [0,0,1,0,1],'g': [0,0,1,1,0],'h': [0,0,1,1,1],
            'i': [0,1,0,0,0],'j': [0,1,0,0,1],'k': [0,1,0,1,0],'l': [0,1,0,1,1],
            'm': [0,1,1,0,0],'n': [0,1,1,0,1],'o': [0,1,1,1,0],'p': [0,1,1,1,1],
            'q': [1,0,0,0,0],'r': [1,0,0,0,1],'s': [1,0,0,1,0],'t': [1,0,0,1,1],
            'u': [1,0,1,0,0],'v': [1,0,1,0,1],'w': [1,0,1,1,0],'x': [1,0,1,1,1],
            'y': [1,1,0,0,0],'z': [1,1,0,0,1],' ': [1,1,0,1,0],' ': [1,1,0,1,1],
            ' ': [1,1,1,0,0],' ': [1,1,1,0,1],' ': [1,1,1,1,0],' ': [1,1,1,1,1],
  }

binToChar = {
            str([0,0,0,0,0]):'a',str([0,0,0,0,1]):'b',str([0,0,0,1,0]):'c',str([0,0,0,1,1]):'d',
            str([0,0,1,0,0]):'e',str([0,0,1,0,1]):'f',str([0,0,1,1,0]):'g',str([0,0,1,1,1]):'h',
            str([0,1,0,0,0]):'i',str([0,1,0,0,1]):'j',str([0,1,0,1,0]):'k',str([0,1,0,1,1]):'l',
            str([0,1,1,0,0]):'m',str([0,1,1,0,1]):'n',str([0,1,1,1,0]):'o',str([0,1,1,1,1]):'p',
            str([1,0,0,0,0]):'q',str([1,0,0,0,1]):'r',str([1,0,0,1,0]):'s',str([1,0,0,1,1]):'t',
            str([1,0,1,0,0]):'u',str([1,0,1,0,1]):'v',str([1,0,1,1,0]):'w',str([1,0,1,1,1]):'x',
            str([1,1,0,0,0]):'y',str([1,1,0,0,1]):'z',str([1,1,0,1,0]):' ',str([1,1,0,1,1]):' ',
            str([1,1,1,0,0]):' ',str([1,1,1,0,1]):' ',str([1,1,1,1,0]):' ',str([1,1,1,1,1]):' '
}

In [0]:
def wordToBin(word):
    wordBin = []
    for c in word.lower():
        wordBin.extend(charTobin[c])
    while len(wordBin) < 20*5:
        wordBin.append(1)
    return torch.tensor(wordBin[0:100],requires_grad=True,dtype=torch.float)

In [0]:
def binToWord(bintensor):
    binList = bintensor.tolist()
    binWord = []
    word = ''
    for i in range(20):
        for j in range(i*5,(i+1)*5):
            binWord.append(int(binList[j]))
        word= word + binToChar[str(binWord)]
        binWord = []
    return word.strip()               

In [0]:
binToWord(wordToBin('test'))

# **NN Model**

In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(100, 256)
        self.fc2 = nn.Linear(256, 100)
        self.fc4 = nn.Linear(100, 100)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc4(x))
        return x


In [0]:
net = Net()

# **Training Loop**

In [0]:
loss_function = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
for epoch in range(1000):
    for data in trainset:
        net.zero_grad()
        inputData = wordToBin(data)
        output = net(inputData)
        loss = loss_function(output.view((1,100)), wordToBin(data).view((1,100)))
        loss.backward()
        optimizer.step()
    print('Loss ==> ', loss.item(),' Epoch ===> ', epoch)

# Test

The aim is to get the output same as the input.

input text is the string given to the ` wordToBin() `. This function converts the string into binary for passing it into the Neural Network.

This network outputs sigmoid values, therefore it has to be converted into binary inorder to convert it to string. Convertion to binary is done by `sig()`, i dont know if there is a builtin function for that, so i made one. This binary is converted to string using the same charector map with `binToWord()`

In [0]:
def sig(x):
    if x>.5:
        return 1
    if x<=.5:
        return 0

In [0]:
inputData = wordToBin('tst')
output = net(inputData)
binToWord(torch.tensor(list(map(sig,output))))

# RoughWork

In [0]:
# charTobin = {
#             'a': 1,'b': 2,'c': 3,'d': 4,
#             'e': 5,'f': 6,'g': 7,'h': 8,
#             'i': 9,'j': 10,'k': 11,'l': 12,
#             'm': 13,'n': 14,'o': 15,'p': 16,
#             'q': 17,'r': 18,'s': 19,'t': 20,
#             'u': 21,'v': 22,'w': 23,'x': 24,
#             'y': 25,'z': 26,' ': 0
# }

# binToChar = {
#             str(1):'a',str(2):'b',str(3):'c',str(4):'d',
#             str(5):'e',str(6):'f',str(7):'g',str(8):'h',
#             str(9):'i',str(10):'j',str(11):'k',str(12):'l',
#             str(13):'m',str(14):'n',str(15):'o',str(16):'p',
#             str(17):'q',str(18):'r',str(19):'s',str(20):'t',
#             str(21):'u',str(22):'v',str(23):'w',str(24):'x',
#             str(25):'y',str(26):'z',str(0):' '
# }